# Eerste Example Functions Flows

This notebook is used to parameterize the .toml configuration file used to run the eerste example as a console application. 

In [1]:
import sys
import tomllib

import numpy as np
import pandas as pd
  
DIRECTORY = '/Users/johnkucharski/Documents/source/functionalflows/'
sys.path.append(DIRECTORY)
from utilities import day_of_water_year, liters_per_day_to_m3_per_second

ModuleNotFoundError: No module named 'utilities'

The functional flows program analyzes water rather than calendar years. The days in the calendar year that correspond with the beginning of the Eerste watershed water year is printed below.

In [ ]:
first_day_of_wy = pd.to_datetime("5-1-2023").day_of_year
print(f'start of water year is the {first_day_of_wy}st day of the year (5-1).')

### Input data

The Eerste example will analyze the input data (df_input) printed to a input.csv file below.

In [ ]:
df_simulated = pd.read_csv(DIRECTORY + '/examples/eerste/simulated_inflow.csv', parse_dates=['dates'], index_col='dates')
df_simulated = df_simulated[['flows']]
df_simulated['flows'] = df_simulated['flows'].replace(-9999.0, np.nan)
df_simulated['flows'] = df_simulated['flows'].apply(liters_per_day_to_m3_per_second)
df_input = df_simulated[['flows']]
df_input.head()


In [ ]:
df_input.to_csv(DIRECTORY + '/examples/eerste/input.csv')
print(f'Input data printed the file at: {DIRECTORY + "/examples/eerste/input.csv"}.')

### Historical data

Functional flow componenents and component metric values are computed based on the historical simulation data below.

In [ ]:
df_historical = pd.read_csv(DIRECTORY + '/examples/eerste/historical_inflow.csv', parse_dates=['date'], index_col='date')
df_historical = df_historical[['historical_inflow']]
df_historical['historical_inflow'] = df_historical['historical_inflow'].replace(-9999.0, np.nan)
df_historical['flow'] = df_historical['historical_inflow'].apply(liters_per_day_to_m3_per_second)
df_historical.dropna(inplace=True)
df_historical.head()

## Dry season baseflow component.

This example component has the following characteristics.
    
* Timing: [01 Dec, 01 May], corresponds with dry season.
* Magnitude: exceeds the 10th percentile of historical period, 1 day flow values.
* Duration: less than 7 days. *NOTE*: this is a *failure* pattern, it identifies periods where timing criteria is met but the magnitude duration is NOT met for 7 or more days. This cooresponds with the timing, magnitude pattern: [1, 0] (i.e. [1=timing metric is met, 0=magnitude threshold is not met]).

To score this component we identify periods in which the flow between 01 Dec - 01 May falls below the 10th percentile of the dry season flow in the historical period for 7 or more days for this. In other words we identify dry season baseflow periods in which the functional flow component critieria is NOT met. This is a *falure* patterns described by the following scoring criteria: [1,0,1]. 

This must be matched with a biotic or abiotic ecosystem function or aquatic community need. For instance, in this case perhaps the needs of large bodied fish or aquatic macroinvertibrates.

In [ ]:
timing = [day_of_water_year(pd.to_datetime("1-1-2023"), start=first_day_of_wy), 366]

df_historical['dry_season'] = df_historical.index.map(lambda t: 1 if timing[0] <= t.day_of_year < timing[1] else 0)
magnitude = df_historical.loc[df_historical['dry_season'] == 1]['flow'].quantile(0.10)

duration = 7
duration_pattern = np.array([1,0])

print(f'timing: {timing} days of water year.')
print(f'magnitude: > {magnitude} cms.')
print(f'duration: > {duration} consecutive days, when the timing duration pattern {duration_pattern}, respectively.')
print(f'scoring criteria: failure pattern [timing=1, magnitude=0, duration=1] (where 1 indicates success and 0 indicates failure to meet the criteria).')

In [ ]:
data = {
    'first_day_of_water_year': first_day_of_wy,
    'components': {
        'dry_season_baseflow': {
            'characteristics': ['timing', 'magnitude', 'duration'],
            'parameters': [[245, 366], [1, 0.011, '>'], [7, [1,0], '>']],
            'scoring_pattern': [1, 0, 1],
            'success_pattern': False
        },
    }
}
with open(DIRECTORY + 'examples/eerste/eerste.toml', 'wb') as f:
    tomllib.dump(data, f)
    #tomli_w.dump(data, f)

## November Pulse Flow

This example component has the following characteristics.
    
* Timing: [01 Nov, 01 Dec], corresponds with a peak in the 7, 14, 30 day flow.
* Magnitude: exceeds the 75th percentile of historical November 1 day flow values.
* Rate of Change: and must represent at least a doubling of the previous days flow value (e.g. it is peak not just a large baseflow).

So, the flow between 01 Nov - 01 Dec must exceed the 75th percentile of daily flow values for November and must represent at least a doubling of the previous 1-day flow value. In this case a *success* pattern: [timing=1, magnitude=1, rate_of_change=1] is scored.

This must be matched with a biotic or abiotic ecosystem function or aquatic community need. For instance, in this case perhaps this flow provides a signal for migratory species.

In [ ]:
timing = [day_of_water_year(pd.to_datetime("11-1-2023"), start=first_day_of_wy), day_of_water_year(pd.to_datetime("12-1-2023"), start=first_day_of_wy)]

df_historical['november'] = df_historical.index.map(lambda t: 1 if timing[0] <= t.day_of_year < timing[1] else 0)
magnitude = df_historical.loc[df_historical['november'] == 1]['flow'].quantile(0.75)

rate_of_change = 2

print(f'timing: {timing} days of water year.')
print(f'magnitude: > {magnitude}.')
print(f'rate_of_change: > {rate_of_change}.')
print(f'scoring_pattern: [1, 1, 1]')

In [ ]:
data = {
    'first_day_of_water_year': first_day_of_wy,
    'components': {
        'dry_season_baseflow': {
            'characteristics': ['timing', 'magnitude', 'duration'],
            'parameters': [[245, 366], [1, 0.011, '>'], [7, [1,0], '>']],
            'scoring_pattern': [1, 0, 1],
            'success_pattern': False
        },
        'november_pulse_flow': {
            'characteristics': ['timing', 'magnitude', 'rate_of_change'],
            'parameters': [[185, 215], [1, 2.256, '>'], [1, 2.0, '>']],
            'scoring_pattern': [1, 1, 1],
            'success_pattern': True 
        },
    }
}
with open(DIRECTORY + 'examples/eerste/eerste.toml', 'wb') as f:
    tomllib.dump(data, f)

## Bankfull flow

This example component has the following characteristics.
    
* Magnitude: is exceeded 50% of time in historic timeseries (2-yr flow).
* Frequency: it must happen at least once every 5 years.

The frequency characteristic metric will be equal to 1 (success) whenever the magnitude criteria has been met at least 1 in the last 5 years. The magnitude metric will only be equal to 1 (success) on days where the flow exceeds the threshold value. Therefore the overall scoring pattern, will only depend on the value of the frequency characteristic metric. This*success* pattern is specified as: [magnitude=\*, frequency=1], where the "\*" indicates that any value is acceptable.

This must be matched with a biotic or abiotic ecosystem function or aquatic community need. For instance, in this case perhaps this flow connects the river with its floodplain, delivering sediment and nutrients to riparian areas adjacent to the river.

In [ ]:
df_historical['wy'] = np.where(df_historical.index.day_of_year < first_day_of_wy, df_historical.index.year - 1, df_historical.index.year)
bankfull_flow = df_historical[['wy', 'flow']].groupby(['wy']).max()['flow'].median()
print(f'The estimated bankfull flow (flow exceeded 50% of the time in the historic time series) is: {bankfull_flow} cms.')

In [ ]:
data = {
    'first_day_of_water_year': first_day_of_wy,
    'components': {
        'dry_season_baseflow': {
            'characteristics': ['timing', 'magnitude', 'duration'],
            'parameters': [[245, 366], [1, 0.011, '>'], [7, [1,0], '>']],
            'scoring_pattern': [1, 0, 1],
            'success_pattern': False
        },
        'november_pulse_flow': {
            'characteristics': ['timing', 'magnitude', 'rate_of_change'],
            'parameters': [[185, 215], [1, 2.256, '>'], [1, 2.0, '>']],
            'scoring_pattern': [1, 1, 1],
            'success_pattern': True 
        },
        'bankfull_flow': {
            'characteristics': ['magnitude', 'frequency'],
            'parameters': [[1, 9.651, '>'], [1, 5, [1], '>']],
            'scoring_pattern': ['*', 1],
            'success_pattern': True
        }
    }
}
with open(DIRECTORY + 'examples/eerste/eerste.toml', 'wb') as f:
    tomllib.dump(data, f)